In [1]:
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.autograd import Variable
from sklearn.metrics import r2_score
from sklearn import preprocessing
#import scipy.io as sio

from models import AE
from models import DNN
import numpy as np
import pandas as pd
import umap


In [2]:
from scipy.stats import pearsonr

# Parameters

In [3]:
# Define parameters
epochs = 500 #200,500,1000
dim_au_in = 11833
dim_au_out = 512 #8, 16, 32, 64, 128, 256,512
dim_dnn_in = dim_au_out
dim_dnn_out=1

# Import data

In [4]:
data_r=pd.read_csv('data/GDSCexpression.csv',index_col=0)
label_r=pd.read_csv('data/GDSClabel.csv',index_col=0)

In [5]:
label_r=label_r.fillna(1)

# Your is gene-cell, mine is cell-gene

In [6]:
len(set(data_r.index))

727

In [7]:
data = data_r
label = label_r.iloc[:,10]
scaler = preprocessing.StandardScaler(with_mean=True, with_std=True)
data = scaler.fit_transform(data)

In [8]:
print(np.std(data))
print(np.mean(data))

0.9999999999999997
6.316789844568889e-19


In [9]:
data.shape

(789, 11833)

In [10]:
label_r.shape

(789, 139)

In [11]:
reducer = umap.UMAP(n_components=dim_au_out)
feature = reducer.fit_transform(data)
feature.shape


(789, 512)

# Split test train

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(feature, label, test_size=0.2, random_state=42)

In [13]:
print(data.shape)
print(label.shape)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(789, 11833)
(789,)
(631, 512) (631,)
(158, 512) (158,)


In [14]:
print(X_train.max())
print(X_train.min())

13.498982
0.29072046


# The model

In [15]:
print(feature.shape)

(789, 512)


In [16]:
feature

array([[ 1.4864289,  6.3059254,  7.277774 , ...,  6.1248293,  4.6318107,
         5.6231327],
       [13.39988  ,  1.7687637,  7.4396744, ...,  6.1940002,  5.0028677,
         5.4401193],
       [ 1.1786144,  5.5773234,  7.5546975, ...,  6.184783 ,  4.7157774,
         5.543639 ],
       ...,
       [ 1.3086456,  5.9235935,  6.8328805, ...,  6.125044 ,  4.607964 ,
         5.657901 ],
       [ 1.7166378,  6.2850018,  6.669288 , ...,  6.08108  ,  4.5422134,
         5.730415 ],
       [ 1.7619338,  5.6065445,  5.98682  , ...,  6.1144233,  4.6758533,
         5.655267 ]], dtype=float32)

In [17]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.1)
clf.fit(X_train, Y_train.values)

Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [18]:
lasso = clf.predict(X_test)

In [19]:
r2_score(lasso,Y_test)

-35.56957650258534

In [20]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(feature)

[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 789 samples in 0.010s...
[t-SNE] Computed neighbors for 789 samples in 0.262s...
[t-SNE] Computed conditional probabilities for sample 789 / 789
[t-SNE] Mean sigma: 0.520990
[t-SNE] KL divergence after 250 iterations with early exaggeration: 49.592804
[t-SNE] KL divergence after 300 iterations: 0.311643


In [21]:
tsne_results

array([[ -7.8325577,  10.74109  ],
       [-16.135729 ,  -4.1019883],
       [ -6.6598253,  18.168083 ],
       ...,
       [ -4.4287763,  13.282541 ],
       [ -4.8203783,   1.8657539],
       [  4.853486 ,   3.9988494]], dtype=float32)

In [22]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.scatter(tsne_results[:, 0], tsne_results[:, 1], label="label")
plt.legend()
plt.savefig("saved/figrues/tsne_umap_ccle.png")

In [23]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)
torch.cuda.set_device(device)

cuda:0


In [24]:
EPOCH = 500

In [25]:
# Load data
# data type conversion
B_feature = torch.FloatTensor(X_train).to(device)
y = torch.FloatTensor(Y_train.values).to(device)
# construct TensorDataset
b_data = TensorDataset(B_feature, y)
trainDataLoader2 = DataLoader(dataset=b_data, batch_size=200, shuffle=False)

In [26]:
b_data.tensors

(tensor([[ 2.1962,  6.6316,  6.1225,  ...,  6.0950,  4.6515,  5.6950],
         [ 1.0000,  5.2908,  6.6875,  ...,  6.1692,  4.7308,  5.6291],
         [12.0107,  1.6760,  7.3024,  ...,  6.2456,  4.8518,  5.5811],
         ...,
         [ 1.6121,  5.4389,  5.9557,  ...,  6.1439,  4.7084,  5.6166],
         [ 2.2726,  7.0952,  6.6600,  ...,  6.0744,  4.5569,  5.7121],
         [12.6368,  1.0704,  7.4475,  ...,  6.2235,  4.9411,  5.5000]],
        device='cuda:0'),
 tensor([4.1360e-03, 2.4651e-01, 1.0000e+00, 1.6831e-01, 2.4553e-01, 4.5574e-02,
         5.6766e-01, 6.5160e-03, 5.1101e-02, 1.0000e+00, 1.0000e+00, 1.3382e-01,
         8.5730e-03, 2.0783e-02, 7.1891e-02, 2.6463e-02, 2.0267e-01, 1.6792e-01,
         2.2742e-01, 5.1173e-01, 5.5870e-02, 9.3305e-02, 3.9400e-03, 9.9018e-02,
         1.0000e+00, 7.3892e-02, 1.1134e-02, 8.2544e-02, 1.1754e-02, 1.4302e-01,
         1.0000e+00, 1.0000e+00, 2.4001e-02, 3.9460e-03, 7.5210e-03, 4.9375e-01,
         7.5744e-02, 1.0000e+00, 3.5260e-03, 5.

In [27]:
b_data

In [28]:
dim_dnn_in

512

In [29]:
# Initialization DNN model

predictor = DNN(dim_dnn_in, dim_dnn_out).to(device)
optimizer = optim.Adam(predictor.parameters(), lr=1e-3,betas=(0.9,0.99))
#loss1-softmax
loss_func = nn.MSELoss().to(device)
#loss2-sigmoid
#loss_func = nn.BCELoss()
#loss3-sigmoid
#loss_func = nn.CrossEntropyLoss()

#criterion = torch.nn.MSELoss(size_average=True)
#criterion = torch.nn.BCELoss(size_average=True) # Defined loss function
#optimizer = optim.Adm(model.parameters(), lr=0.01) # Defined optimizer
loss_train = np.zeros((epochs, 1))

In [30]:
# train model
for epoch in range(EPOCH):
    print('Epoch: ',epoch)
    for step,(batch_x,batch_y) in enumerate(trainDataLoader2):
        b_x = Variable(batch_x)
        b_y = Variable(batch_y)
        # predict label
        output = predictor(b_x)
        # b_y=F.sigmoid(b_y) 
        
        #print
        #print(output)
        #print(b_y)
        # compute loss
        loss = loss_func(output,b_y)
        #loss = criterion(output, b_y)
        
        # update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    loss_train[epoch,0] = loss.item()  
    print('Epoch: %04d, Training loss=%.8f' %
          (epoch+1, loss.item())) 


Epoch:  0


D:\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:445: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
D:\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:445: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
D:\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:445: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=se

Epoch: 0001, Training loss=0.11799109
Epoch:  1


D:\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:445: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
D:\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:445: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
D:\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:445: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=se

Epoch: 0002, Training loss=0.12254722
Epoch:  2


...

Epoch: 0498, Training loss=0.10533340
Epoch:  498


D:\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:445: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
D:\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:445: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
D:\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:445: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=se

Epoch: 0499, Training loss=0.10533643
Epoch:  499


D:\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:445: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
D:\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:445: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
D:\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:445: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=se

Epoch: 0500, Training loss=0.10533272


In [31]:
testFeature = torch.FloatTensor(X_test).to(device)


In [32]:
testpredict = predictor(testFeature)

In [33]:
testpredict

tensor([[0.2204],
        [0.2198],
        [0.2191],
        [0.2187],
        [0.2148],
        [0.2190],
        [0.2206],
        [0.2189],
        [0.2187],
        [0.2195],
        [0.2196],
        [0.2237],
        [0.2233],
        [0.2190],
        [0.2194],
        [0.2233],
        [0.2196],
        [0.2197],
        [0.2192],
        [0.2233],
        [0.2195],
        [0.2235],
        [0.2203],
        [0.2205],
        [0.2236],
        [0.2234],
        [0.2195],
        [0.2196],
        [0.2191],
        [0.2205],
        [0.2188],
        [0.2195],
        [0.2236],
        [0.2192],
        [0.2195],
        [0.2187],
        [0.2204],
        [0.2194],
        [0.2189],
        [0.2190],
        [0.2200],
        [0.2191],
        [0.2186],
        [0.2194],
        [0.2196],
        [0.2193],
        [0.2232],
        [0.2198],
        [0.2197],
        [0.2233],
        [0.2197],
        [0.2198],
        [0.2200],
        [0.2193],
        [0.2195],
        [0

In [34]:
from sklearn.metrics import mean_squared_error

In [35]:
r2_score(testpredict.detach().cpu().numpy(),Y_test)

-53743.37025956225

In [36]:
mean_squared_error(testpredict.detach().cpu().numpy(),Y_test)

0.13222994598430116